In [22]:
import yfinance as yf
from financetoolkit import Toolkit
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from path import Path

In [23]:
# Get staements
api_key = 'DhSKlDzkmluzGeZ4mK9mvI0lQDUDmRpa'

#get data
companies = Toolkit(
    tickers=['ACLS'],
    api_key=api_key,
)

balance_sheet = companies.get_balance_sheet_statement()
income_statement = companies.get_income_statement()
cash_flow_statement = companies.get_cash_flow_statement()



Obtaining cashflow data: 100%|██████████| 1/1 [00:00<00:00,  9.76it/s]


In [24]:
# Make a df with values that need to be forecasted to do necessary accounting to get fcf


In [25]:
# Linear Regression
X = cash_flow_statement.drop(columns=['2023'])
y = cash_flow_statement['2023']

In [26]:
# Create a model with scikit-learn
model = LinearRegression()

In [27]:
# Fit the data into the model
model.fit(X, y)

LinearRegression()

In [28]:
# Make predictions using the X set
predicted_y_values = model.predict(X)

In [29]:
# Create a copy of the original data
cash_flow_statement_forecasting = cash_flow_statement.copy()

# Add a column with the predicted salary values
cash_flow_statement_forecasting["2024"] = predicted_y_values



In [30]:
# Balance sheet 'Total Debt'
# Balance sheet 'Total Equity'
# total capital = short term debt + long term debt + shareholders equity
# after tax cost of debt = interest expense - tax shield 

# Calculate Total capital and declare necessary variables
total_capital = balance_sheet['2023'].loc['Short Term Debt']+balance_sheet['2023'].loc['Long Term Debt']+balance_sheet['2023'].loc['Total Shareholder Equity']
bt_cod = (income_statement['2023'].loc['Interest Expense']/balance_sheet['2023'].loc['Total Debt'])*100 
tax_rate = balance_sheet['2023'].loc['Tax Payables']/income_statement['2023'].loc['Income Before Tax']
at_cod = (bt_cod*(1-tax_rate))/balance_sheet['2023'].loc['Total Debt']
rf = 0.04
beta = 1.64 # 5yr monthly
avg_return = 0.08
rp = avg_return-rf
erp = rf + beta * (rp-rf)
cost_of_equity = rf + beta *(erp - rf)
inital_value = cash_flow_statement['2020'].loc['Free Cash Flow']
ending_value= cash_flow_statement['2023'].loc['Free Cash Flow']
n = 5

# Calculate WACC
wacc = ((balance_sheet['2023'].loc['Total Debt']/total_capital)*at_cod)+((balance_sheet['2023'].loc['Total Equity']/total_capital)*cost_of_equity)

In [33]:
# Terminal Value = (ending_value * (1+growth_rate))/(wacc - growth_rate)
# Growth Rate
growth_rate = (ending_value/inital_value)**(1/n)-1
tv = (ending_value*(1 + growth_rate))/(wacc-growth_rate)
dcf = 





In [34]:
dcf_metrics = pd.DataFrame(tv)

ValueError: DataFrame constructor not properly called!